In [ ]:
from dotenv import load_dotenv

load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2

!export PYTHONPATH=":./python"

In [6]:
from enum import Enum


class LlmCache(str, Enum):
    MEMORY = "memory"
    SQLITE = "sqlite"
    NONE = "none"
    # Todo : Add Postgres, Similarity, ....

    @classmethod
    def from_value(cls, value):
        for member in cls:
            if member.value.lower() == value.lower():
                return member
        return None

    @classmethod
    def values(cls) -> list[str]:
        return [member.value for member in cls]

In [7]:
LlmCache.values()

['memory', 'sqlite', 'none']